<a href="https://colab.research.google.com/github/yinpu/deep_learning_study/blob/master/MFFRM(without_KGE).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MFFRM(without KGE)
**去掉知识图谱部分**

## 数据预处理
* book -->book-crossing数据集
* music -->last.fm数据集
* movie -->movielen数据集

In [2]:
#数据预处理函数
import numpy as np
import os
import  tensorflow as tf

RATING_FILE_NAME={
    'movie': 'ratings.dat',
    'book':'BX-Book-Ratings.csv',
    'music':'user_artists.dat'
}
SEP={
    'movie':'::',
    'book':';',
    'music':'\t'
}
THRESHOLD={
    'movie':4,
    'book':0,
    'music':0
}
DATASETS={'movie','book','music'}

def read_item_index2entity_id_file():
    file = './data/'+DATASET+'/item_index2entity_id.txt'
    i=0
    for line in open(file,encoding='utf-8').readlines():
        item_index=line.strip().split('\t')[0]
        satori_id=line.strip().split('\t')[1]
        item_index_old2new[item_index]=i
        entity_id2index[satori_id]=i
        i+=1
        
def convert_rating():
    file = './data/'+DATASET+'/'+RATING_FILE_NAME[DATASET]
    item_set=set(item_index_old2new.values())
    user_pos_rating={}
    user_neg_rating={}
    for line in open(file,encoding='utf-8').readlines():
        array =line.strip().split(SEP[DATASET])
        if DATASET=='book':
            array=list(map(lambda x:x[1:-1],array))
        item_index_old=array[1]
        if item_index_old not in item_index_old2new:
            continue
        item_index=item_index_old2new[item_index_old]
        user_index_old=int(array[0])
        rating=float(array[2])
        if rating >=THRESHOLD[DATASET]:
            if user_index_old not in user_pos_rating:
                user_pos_rating[user_index_old]=set()
            user_pos_rating[user_index_old].add(item_index)
        else:
            if user_index_old not in user_neg_rating:
                user_neg_rating[user_index_old]=set()
            user_neg_rating[user_index_old].add(item_index)
    print('coverting rating file . . .')
    writer=open('data/'+DATASET+'/rating_final.txt','w',encoding='utf-8')
    user_cnt=0
    user_index_old2new={}
    for user_index_old,pos_item_set in user_pos_rating.items():
        if user_index_old not in user_index_old2new:
            user_index_old2new[user_index_old]=user_cnt
            user_cnt+=1
        user_index=user_index_old2new[user_index_old]
        for item in pos_item_set:
            writer.write("%d\t%d\t1\n"%(user_index,item))
        unwatch_set=item_set-pos_item_set
        if user_index_old in user_neg_rating:
            unwatch_set-=user_neg_rating[user_index_old]
        for item in np.random.choice(list(unwatch_set),size=len(pos_item_set),replace=False):
            writer.write("%d\t%d\t0\n"%(user_index,item))
    writer.close()
    print("num of users:",user_cnt)
    print("num of item:",len(item_set))

def convert_kg():
    print('converting kg.txt file')
    entity_cnt=len(entity_id2index)
    relation_cnt=0
    writer=open('data/'+DATASET+'/kg_final.txt','w',encoding='utf-8')
    file=open('data/'+DATASET+'/kg.txt','r',encoding='utf-8')
    for line in file:
        array=line.strip().split('\t')
        head_old=array[0]
        relation_old=array[1]
        tail_old=array[2]
        if head_old not in entity_id2index:
            continue
        head=entity_id2index[head_old]
        if tail_old not in entity_id2index:
            entity_id2index[tail_old]=entity_cnt
            entity_cnt+=1
        tail=entity_id2index[tail_old]
        if relation_old not in relation_id2index:
            relation_id2index[relation_old]=relation_cnt
            relation_cnt+=1
        relation=relation_id2index[relation_old]
        writer.write("%d\t%d\t%d\n"%(head,relation,tail))
    writer.close()
    print('num of entities:',entity_cnt)
    print('num of relations:',relation_cnt)

In [3]:
from google.colab import drive
import os
drive.mount("/content/drive")
os.chdir('/content/drive/Shared drives/yinpu/MyProject/FFMKR')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
#数据集处理
for DATASET in DATASETS:
    print(DATASET+"  :   ")
    np.random.seed(555)
    entity_id2index={}
    relation_id2index={}
    item_index_old2new={}
    read_item_index2entity_id_file()
    convert_rating()
    convert_kg()

movie  :   
coverting rating file . . .
num of users: 6036
num of item: 2347
converting kg.txt file
num of entities: 6729
num of relations: 7
music  :   
coverting rating file . . .
num of users: 1872
num of item: 3846
converting kg.txt file
num of entities: 9366
num of relations: 60
book  :   
coverting rating file . . .
num of users: 17860
num of item: 14910
converting kg.txt file
num of entities: 24039
num of relations: 10


In [49]:
def model(n_user: int, n_item: int, dim=8, L=1,H=1, l2=1e-6) -> tf.keras.Model:
    l2=tf.keras.regularizers.l2(l2)
    user_id=tf.keras.Input(shape=(),dtype=tf.int32,name='user_id')
    item_id=tf.keras.Input(shape=(),dtype=tf.int32,name='item_id')
    # head_id=tf.keras.Input(shape=(),dtype=tf.int32,name='head_id')
    # relation_id=tf.keras.Input(shape=(),dtype=tf.int32,name='relation_id')
    # tail_id=tf.keras.Input(shape=(),dtype=tf.int32,name='tail_id')
    user_embedding=tf.keras.layers.Embedding(input_dim=n_user,output_dim=dim,
                                             embeddings_regularizer=l2)
    item_embedding=tf.keras.layers.Embedding(input_dim=n_item,output_dim=dim,
                                             embeddings_regularizer=l2)
    # entity_embedding=tf.keras.layers.Embedding(input_dim=n_entity,output_dim=dim,
    #                                            embeddings_regularizer=l2)
    # relation_embedding=tf.keras.layers.Embedding(input_dim=n_relation,output_dim=dim,
    #                                            embeddings_regularizer=l2)
    u=user_embedding(user_id)
    i=item_embedding(item_id)
    # h=entity_embedding(head_id)
    # t=entity_embedding(tail_id)
    # r=relation_embedding(relation_id)
    for _ in range(L):
        u=tf.keras.layers.Dense(dim,activation=tf.keras.activations.relu,kernel_regularizer=l2)(u)
        i=tf.keras.layers.Dense(dim,activation=tf.keras.activations.relu,kernel_regularizer=l2)(i)
        # i,h=feature_fusion_unit(inputs=(i,h),weight_regularizer=l2)
        # t=tf.keras.layers.Dense(dim,activation=tf.keras.activations.relu,kernel_regularizer=l2)(t)
    rs=tf.keras.activations.relu(u*i)
    for _ in range(H-1):
        rs=tf.keras.layers.Dense(dim,activation=tf.keras.activations.relu,kernel_regularizer=l2)(rs)
    rs=tf.keras.layers.Dense(1,activation=tf.keras.activations.sigmoid,kernel_regularizer=l2)(rs)
    return tf.keras.Model(inputs=[user_id,item_id,head_id],outputs=rs)

In [50]:
def load_data(DATASET):
    n_user,n_item,train_data,eval_data,test_data=load_rating(DATASET)
    n_entity,n_relation,kg=load_kg(DATASET)
    print('data loaded')
    return n_user,n_item,n_entity,n_relation,train_data,eval_data,test_data,kg

def load_rating(DATASET):
    print('reading rating file ....')
    rating_file='data/'+DATASET+'/rating_final'
    if os.path.exists(rating_file+'.npy'):
        rating_np=np.load(rating_file+'.npy')
    else:
        rating_np=np.loadtxt(rating_file+'.txt',dtype=np.int32)
        np.save(rating_file+'.npy',rating_np)
    n_user=len(set(rating_np[:,0]))
    n_item=len(set(rating_np[:,1]))
    train_data,eval_data,test_data=dataset_split(rating_np)
    return n_user,n_item,train_data,eval_data,test_data

def dataset_split(rating_np):
    print('splitting dataset ...')
    eval_ratio=0.2
    test_ration=0.2
    n_rating=rating_np.shape[0]
    eval_indices=np.random.choice(list(range(n_rating)),size=int(n_rating*eval_ratio),replace=False)
    left=set(range(n_rating))-set(eval_indices)
    test_indices=np.random.choice(list(left),size=int(n_rating*test_ration),replace=False)
    train_indices=list(left-set(test_indices))
    train_data,eval_data,test_data=rating_np[train_indices],rating_np[eval_indices],rating_np[test_indices]
    return train_data,eval_data,test_data
def load_kg(DATASET):
    print('reading KG file')
    kg_file='data/'+DATASET+'/kg_final'
    if os.path.exists(kg_file + '.npy'):
        kg = np.load(kg_file + '.npy')
    else:
        kg = np.loadtxt(kg_file + '.txt', dtype=np.int32)
        np.save(kg_file + '.npy', kg)
    n_entity=len(set(kg[:,0])|set(kg[:,2]))
    n_relation=len(set(kg[:,1]))
    return n_entity,n_relation,kg

In [51]:
def preprocess_data(data):
    u = tf.constant(data[:, 0], dtype=tf.int32)
    i = tf.constant(data[:, 1], dtype=tf.int32)
    h = tf.constant(data[:, 1], dtype=tf.int32)
    r = tf.constant(data[:, 2], dtype=tf.float32)
    return {'user_id': u, 'item_id': i}, r

def train(DATASET, DEFAULT):
    n_user, n_item, n_entity, n_relation, train_data, eval_data, test_data, kg =load_data(DATASET)
    rs_model = model(n_user, n_item,dim=DEFAULT[DATASET]['dim'])
    epochs = DEFAULT[DATASET]['epochs']
    batch = DEFAULT[DATASET]['batch_size']
    train_ds = tf.data.Dataset.from_tensor_slices(preprocess_data(train_data))
    train_ds = train_ds.shuffle(len(train_data)).batch(batch)
    eval_ds = tf.data.Dataset.from_tensor_slices(preprocess_data(eval_data))
    eval_ds =eval_ds.batch(batch)
    rs_model.compile(optimizer=tf.optimizers.Adam( DEFAULT[DATASET]['lr_rs']), loss=tf.keras.losses.binary_crossentropy,
                     metrics=['AUC', tf.keras.metrics.binary_accuracy])
    for epoch in range(epochs):
        history=rs_model.fit(train_ds, epochs=epoch + 1, validation_data=eval_ds, verbose=1, initial_epoch=epoch)
    return rs_model

In [52]:
DEFAULT={
        'movie': {'epochs': 10, 'dim': 8, 'L': 1, 'H': 1, 'batch_size': 4096, 'lr_rs': 0.02, 'lr_kg': 0.01,
                  'kge_interval': 3},
        'book': {'epochs': 10, 'dim': 8, 'L': 1, 'H': 1, 'batch_size': 32, 'lr_rs': 2e-4, 'lr_kg': 2e-5,
                 'kge_interval': 2},
        'music': {'epochs': 30, 'dim': 4, 'L': 2, 'H': 1, 'batch_size': 256, 'lr_rs': 1e-4, 'lr_kg': 2e-4,
                  'kge_interval': 2}
    }
DATASET='movie'
rs_model=train(DATASET,DEFAULT)

reading rating file ....
splitting dataset ...
reading KG file
data loaded


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


111/111 [==============================] - 2s 20ms/step - loss: 0.4538 - auc: 0.8686 - binary_accuracy: 0.7915 - val_loss: 0.4257 - val_auc: 0.8851 - val_binary_accuracy: 0.8032
Epoch 2/2
111/111 [==============================] - 2s 18ms/step - loss: 0.4155 - auc: 0.8906 - binary_accuracy: 0.8077 - val_loss: 0.4260 - val_auc: 0.8860 - val_binary_accuracy: 0.8034
Epoch 3/3
111/111 [==============================] - 2s 18ms/step - loss: 0.4056 - auc: 0.8969 - binary_accuracy: 0.8144 - val_loss: 0.4257 - val_auc: 0.8876 - val_binary_accuracy: 0.8075
Epoch 4/4
111/111 [==============================] - 2s 18ms/step - loss: 0.3868 - auc: 0.9086 - binary_accuracy: 0.8269 - val_loss: 0.4111 - val_auc: 0.8980 - val_binary_accuracy: 0.8192
Epoch 5/5
111/111 [==============================] - 2s 18ms/step - loss: 0.3591 - auc: 0.9238 - binary_accuracy: 0.8446 - val_loss: 0.3998 - val_auc: 0.9063 - val_binary_accuracy: 0.8271
Epoch 6/6
111/111 [==============================] - 2s 18ms/step - lo

In [53]:
n_user, n_item, n_entity, n_relation, train_data, eval_data, test_data, kg =load_data(DATASET)
result=rs_model.evaluate([test_data[:,0],test_data[:,1]],test_data[:,2])

reading rating file ....
splitting dataset ...
reading KG file
data loaded
4712/4712 [==============================] - 17s 4ms/step - loss: 0.3264 - auc: 0.9425 - binary_accuracy: 0.8720


In [54]:
print("在测试集上auc："+str(result[1]))
print("在测试集上acc："+str(result[2]))

在测试集上auc：0.9425419569015503
在测试集上acc：0.8720232844352722


In [69]:
DEFAULT={
        'movie': {'epochs': 10, 'dim': 8, 'L': 1, 'H': 1, 'batch_size': 4096, 'lr_rs': 0.02, 'lr_kg': 0.01,
                  'kge_interval': 3},
        'book': {'epochs': 2, 'dim': 8, 'L': 1, 'H': 1, 'batch_size': 32, 'lr_rs': 0.001, 'lr_kg': 2e-5,
                 'kge_interval': 2},
        'music': {'epochs': 30, 'dim': 4, 'L': 2, 'H': 1, 'batch_size': 256, 'lr_rs': 1e-4, 'lr_kg': 2e-4,
                  'kge_interval': 2}
    }
DATASET='book'
rs_model=train(DATASET,DEFAULT)

reading rating file ....
splitting dataset ...
reading KG file
data loaded


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


2621/2621 [==============================] - 19s 7ms/step - loss: 0.6065 - auc: 0.7029 - binary_accuracy: 0.6604 - val_loss: 0.5809 - val_auc: 0.7329 - val_binary_accuracy: 0.6984
Epoch 2/2
2621/2621 [==============================] - 19s 7ms/step - loss: 0.5191 - auc: 0.8153 - binary_accuracy: 0.7394 - val_loss: 0.5980 - val_auc: 0.7360 - val_binary_accuracy: 0.6920


In [70]:
n_user, n_item, n_entity, n_relation, train_data, eval_data, test_data, kg =load_data(DATASET)
result=rs_model.evaluate([test_data[:,0],test_data[:,1]],test_data[:,2])

reading rating file ....
splitting dataset ...
reading KG file
data loaded
874/874 [==============================] - 3s 4ms/step - loss: 0.4989 - auc: 0.8409 - binary_accuracy: 0.7608


In [71]:
print("在测试集上auc："+str(result[1]))
print("在测试集上acc："+str(result[2]))

在测试集上auc：0.8408634066581726
在测试集上acc：0.760778546333313


In [81]:
DEFAULT={
        'movie': {'epochs': 10, 'dim': 8, 'L': 1, 'H': 1, 'batch_size': 4096, 'lr_rs': 0.02, 'lr_kg': 0.01,
                  'kge_interval': 3},
        'book': {'epochs': 3, 'dim': 8, 'L': 1, 'H': 1, 'batch_size': 32, 'lr_rs': 0.001, 'lr_kg': 2e-5,
                 'kge_interval': 2},
        'music': {'epochs':4, 'dim': 8, 'L': 1, 'H': 1, 'batch_size': 256, 'lr_rs': 1e-3, 'lr_kg': 2e-4,
                  'kge_interval': 2}
    }
DATASET='music'
rs_model=train(DATASET,DEFAULT)

reading rating file ....
splitting dataset ...
reading KG file
data loaded


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


100/100 [==============================] - 1s 10ms/step - loss: 0.6718 - auc: 0.6807 - binary_accuracy: 0.6254 - val_loss: 0.6054 - val_auc: 0.7725 - val_binary_accuracy: 0.7272
Epoch 2/2
100/100 [==============================] - 1s 7ms/step - loss: 0.5211 - auc: 0.8158 - binary_accuracy: 0.7599 - val_loss: 0.5282 - val_auc: 0.7969 - val_binary_accuracy: 0.7492
Epoch 3/3
100/100 [==============================] - 1s 7ms/step - loss: 0.4557 - auc: 0.8674 - binary_accuracy: 0.7906 - val_loss: 0.5482 - val_auc: 0.7971 - val_binary_accuracy: 0.7422
Epoch 4/4
100/100 [==============================] - 1s 7ms/step - loss: 0.4236 - auc: 0.8877 - binary_accuracy: 0.8066 - val_loss: 0.5726 - val_auc: 0.7944 - val_binary_accuracy: 0.7254


In [82]:
n_user, n_item, n_entity, n_relation, train_data, eval_data, test_data, kg =load_data(DATASET)
result=rs_model.evaluate([test_data[:,0],test_data[:,1]],test_data[:,2])

reading rating file ....
splitting dataset ...
reading KG file
data loaded
265/265 [==============================] - 1s 4ms/step - loss: 0.4571 - auc: 0.8670 - binary_accuracy: 0.7941


In [83]:
print("在测试集上auc："+str(result[1]))
print("在测试集上acc："+str(result[2]))

在测试集上auc：0.8670268654823303
在测试集上acc：0.7940725088119507
